<a href="https://colab.research.google.com/github/jungeun919/Pytorch_study/blob/main/Pytorch_Tutorial/Build_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 신경망 구축

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
# torch.cuda를 사용할 수 있는 확인
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using {} device".format(device))

Using cpu device


# 클래스 정의

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [5]:
# 모델을 사용하기 위해 입력 데이터를 전달
# 모델의 forward는 일부 백그라운드 작업과 함께 실행
# nn.Softmax 모듈의 인스턴스를 통해 예측 확률을 얻음
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([9])


# 모델 레이어

In [6]:
# 28x28 크기의 이미지 3개로 구성된 sample minibatch를 통해
# 네트워크를 통과할 때 어떻게 변화되는지 관찰
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten

In [7]:
# 레이어를 초기화하여 28x28 이미지를 784 픽셀 값의 연속 배열로 변환
# minibatch 차원(dim=0)이 유지됨
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear

In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU

In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.3124, -0.2604,  0.0015,  0.4191, -0.7274,  0.0919,  0.2043, -0.2015,
          0.3797,  0.5032, -0.0997,  0.3010,  0.0109,  0.1927, -0.3203, -0.4417,
          0.3473, -0.4110,  0.5386,  0.1591],
        [ 0.1900,  0.2311, -0.0347,  0.0456, -0.4166,  0.0210, -0.1937, -0.0599,
          0.4571,  0.5273, -0.0536,  0.5482, -0.2114,  0.6159, -0.3757, -0.2620,
          0.0684, -0.2726,  0.4253, -0.0046],
        [ 0.4886, -0.2812,  0.0412,  0.1820, -0.5407,  0.2026,  0.3431,  0.1188,
          0.6419,  0.1881, -0.2943,  0.0508,  0.0628,  0.4677, -0.3121, -0.1955,
          0.0794, -0.2063,  0.6513,  0.0589]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.3124, 0.0000, 0.0015, 0.4191, 0.0000, 0.0919, 0.2043, 0.0000, 0.3797,
         0.5032, 0.0000, 0.3010, 0.0109, 0.1927, 0.0000, 0.0000, 0.3473, 0.0000,
         0.5386, 0.1591],
        [0.1900, 0.2311, 0.0000, 0.0456, 0.0000, 0.0210, 0.0000, 0.0000, 0.4571,
         0.5273, 0.0000, 0.5482, 0.0000, 0.6159, 0.000

### nn.Sequential

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)

input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

### nn.Softmax

In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

# 모델 매개변수 (Model Paremeters)

- nn.Module은 모델 객체 내의 정의된 모든 필드를 자동으로 추적
- parameters() 또는 named_parameters() 메서드를 사용하여 모든 매개변수에 엑세스 할 수 있음

In [12]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[ 0.0236,  0.0314,  0.0027,  ...,  0.0139,  0.0291, -0.0346],
        [-0.0141,  0.0280, -0.0091,  ..., -0.0279, -0.0060,  0.0104]],
       grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([0.0326, 0.0257], grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[ 0.0289, -0.0144, -0.0229,  ...,  0.0259,  0.0423,  0.0353],
        [-0.0294, -0.0333, -0.0049,  ...,  0.0426, -0.0441,  0.0117]],
       grad_fn=<SliceBackward>) 

Layer: linear_relu_stack